In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms

In [3]:
import os
from glob import glob
import random
import time
import datetime
os.environ['KERAS_BACKEND'] = 'tensorflow'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # 3 = INFO, WARNING, and ERROR messages are not printed

from tqdm import tqdm

import numpy as np
import pandas as pd
from IPython.display import FileLink
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns 
%matplotlib inline
from IPython.display import display, Image
import matplotlib.image as mpimg
import cv2

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_files       
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.metrics import log_loss

from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.applications.vgg16 import VGG16

#our additions
import json

Using TensorFlow backend.


In [4]:
# Load the dataset previously downloaded from Kaggle
NUMBER_CLASSES = 10

def load_train():
    start_time = time.time()
    train_pose = [] 
    train_labels = []
    # Loop over the training folder 
    for classed in tqdm(range(NUMBER_CLASSES)):
        print('Loading directory c{}'.format(classed))
        files = glob(os.path.join('sample_json', 'c' + str(classed), '*.json'))
        for file in files:
            jsonFile = open(file, 'r')
            try: 
                values = json.load(jsonFile)['people'][0]['pose_keypoints_2d']
            except IndexError: 
                continue
            #print(values)
            #print(len(values))
            train_pose.append(values)
            train_labels.append(classed)
    print("Data Loaded in {} second".format(time.time() - start_time))
    return train_pose, train_labels 

In [5]:
train_pose, train_labels = load_train()
# train_labels            = np_utils.to_categorical(train_labels, 10)

  0%|          | 0/10 [00:00<?, ?it/s]

Loading directory c0


 10%|█         | 1/10 [00:08<01:17,  8.61s/it]

Loading directory c1


 20%|██        | 2/10 [00:16<01:06,  8.31s/it]

...

Loading directory c8


 90%|█████████ | 9/10 [01:09<00:07,  7.27s/it]

Loading directory c9


100%|██████████| 10/10 [01:16<00:00,  7.63s/it]


Data Loaded in 76.31291675567627 second


In [6]:
print(len(train_pose))
print(len(train_labels))

22394
22394


In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(train_pose, train_labels, test_size= 0.2, random_state=42)

In [37]:
print((Y_train))

[5, 7, 3, 4, 2, 1, 2, 3, 0, 8, 4, 6, 3, 8, 4, 8, 9, 0, 3, 2, 0, 2, 8, 4, 8, 4, 5, 6, 1, 0, 3, 5, 9, 6, 4, 8, 7, 0, 2, 1, 3, 5, 8, 6, 7, 5, 2, 9, 9, 5, 1, 5, 0, 9, 4, 1, 9, 8, 1, 4, 9, 6, 3, 3, 4, 5, 4, 3, 7, 2, 3, 6, 3, 2, 8, 0, 0, 1, 2, 7, 7, 9, 2, 1, 2, 2, 8, 9, 0, 0, 1, 8, 4, 6, 3, 8, 7, 3, 8, 4, 3, 7, 7, 3, 9, 0, 3, 8, 3, 2, 3, 2, 1, 7, 4, 6, 9, 2, 3, 4, 8, 5, 7, 7, 8, 5, 1, 5, 4, 2, 3, 7, 2, 4, 9, 4, 0, 7, 5, 5, 5, 5, 6, 1, 6, 3, 6, 3, 1, 6, 6, 8, 9, 4, 9, 3, 3, 6, 2, 9, 5, 4, 3, 1, 2, 7, 6, 4, 2, 8, 5, 8, 6, 3, 2, 1, 5, 4, 7, 7, 6, 4, 3, 3, 7, 8, 0, 6, 3, 7, 0, 2, 6, 0, 2, 3, 6, 4, 3, 2, 7, 4, 1, 7, 2, 3, 9, 0, 5, 6, 8, 3, 0, 3, 4, 1, 6, 6, 3, 7, 5, 7, 4, 3, 2, 8, 3, 0, 2, 8, 7, 6, 7, 8, 7, 0, 0, 4, 9, 1, 9, 4, 3, 8, 6, 5, 5, 9, 4, 6, 0, 5, 9, 6, 1, 0, 1, 4, 5, 3, 3, 1, 2, 6, 9, 9, 5, 2, 5, 8, 8, 8, 2, 2, 7, 5, 6, 1, 2, 9, 7, 8, 6, 6, 6, 5, 9, 9, 3, 7, 1, 5, 6, 0, 6, 6, 1, 3, 6, 6, 1, 2, 0, 9, 9, 2, 3, 4, 6, 9, 6, 8, 8, 0, 4, 6, 7, 8, 2, 5, 6, 2, 3, 4, 7, 9, 1, 7, 2, 9, 2, 6, 2, 

In [44]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.hidden = nn.Linear(75, 50)
        self.hidden2 = nn.Linear(50, 25)
        self.output = nn.Linear(25, 10)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.hidden(x)
        x = self.sigmoid(x)
        x = self.hidden2(x)
        x = self.sigmoid(x)
        x = self.output(x)
        return x


In [88]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hidden = nn.Linear(75, 150)
        self.hidden2 = nn.Linear(150, 100)
        self.hidden3 = nn.Linear(100, 30)
        self.output = nn.Linear(30, 10)
        self.sigmoid = nn.Sigmoid()
        self.tanh    = nn.Tanh() 
        
    def forward(self, x):
        x = self.hidden(x)
        x = self.sigmoid(x)
        x = self.hidden2(x)
        x = self.tanh(x)
        x = self.hidden3(x)
        x = F.tanh(x)
        x = self.output(x)
        return x
    


In [89]:
net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=.5)

In [90]:
batch_size = 50
for epoch in range(30):  # loop over the dataset multiple times
    running_loss = 0.0
    for i in range(0, len(X_train), batch_size):
        # get the inputs; data is a list of [inputs, labels]
        # print(i)
        inputs = torch.FloatTensor(X_train[i:i+50])
        labels = torch.LongTensor(Y_train[i:i+50])
        
        # print(labels)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 4000 == 0 and i>10:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  4001] loss: 0.093
[1,  8001] loss: 0.092
[1, 12001] loss: 0.092
[1, 16001] loss: 0.092
[2,  4001] loss: 0.092
[2,  8001] loss: 0.091
[2, 12001] loss: 0.091
[2, 16001] loss: 0.091
[3,  4001] loss: 0.092
[3,  8001] loss: 0.091
[3, 12001] loss: 0.091
[3, 16001] loss: 0.091
[4,  4001] loss: 0.091
[4,  8001] loss: 0.090
[4, 12001] loss: 0.090
[4, 16001] loss: 0.090
[5,  4001] loss: 0.091
[5,  8001] loss: 0.089
[5, 12001] loss: 0.089
[5, 16001] loss: 0.089
[6,  4001] loss: 0.090
[6,  8001] loss: 0.088
[6, 12001] loss: 0.088
[6, 16001] loss: 0.088
[7,  4001] loss: 0.089
[7,  8001] loss: 0.087
[7, 12001] loss: 0.087
[7, 16001] loss: 0.087
[8,  4001] loss: 0.087
[8,  8001] loss: 0.085
[8, 12001] loss: 0.085
[8, 16001] loss: 0.084
[9,  4001] loss: 0.084
[9,  8001] loss: 0.082
[9, 12001] loss: 0.082
[9, 16001] loss: 0.081
[10,  4001] loss: 0.081
[10,  8001] loss: 0.078
[10, 12001] loss: 0.078
[10, 16001] loss: 0.077
[11,  4001] loss: 0.077
[11,  8001] loss: 0.075
[11, 12001] loss: 0.074
[11,

In [91]:
outputs = net(torch.FloatTensor(X_test))
_, predicted = torch.max(outputs, 1)

In [48]:
classes = ('Safe driving', 'Texting - right', 'Talking on the phone - right', 'Texting - left',
           'Talking on the phone - left', 'Operating the radio', 'Drinking', 'Reaching Behind', 'Hair and makeup', 'Talking to Passenger')
print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(100)))

Predicted:  Talking on the phone - right Talking on the phone - right Safe driving Safe driving Safe driving Drinking Safe driving Talking on the phone - right Talking on the phone - right Safe driving Talking on the phone - right Talking on the phone - right Safe driving Talking on the phone - right Safe driving Talking on the phone - right Talking on the phone - right Drinking Talking on the phone - right Talking on the phone - right Safe driving Safe driving Talking on the phone - right Safe driving Operating the radio Safe driving Talking on the phone - left Safe driving Safe driving Operating the radio Talking on the phone - right Operating the radio Safe driving Drinking Drinking Safe driving Talking on the phone - right Talking on the phone - right Talking on the phone - right Talking on the phone - right Talking on the phone - right Drinking Safe driving Safe driving Talking to Passenger Talking on the phone - left Safe driving Talking on the phone - right Texting - left Talkin

In [92]:
count = 0
for i in range(len(Y_test)):
    if Y_test[i] == predicted[i]:
        count+=1
print(count/len(Y_test))


0.7450323732976111


In [95]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

for i in range(len(Y_test)):
    label = Y_test[i]
    if Y_test[i] == predicted[i]:
        class_correct[label] += 1
    class_total[label] += 1
for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of Safe driving : 80 %
Accuracy of Texting - right : 86 %
Accuracy of Talking on the phone - right : 87 %
Accuracy of Texting - left : 86 %
Accuracy of Talking on the phone - left : 80 %
Accuracy of Operating the radio : 91 %
Accuracy of Drinking : 37 %
Accuracy of Reaching Behind : 97 %
Accuracy of Hair and makeup : 16 %
Accuracy of Talking to Passenger : 72 %


In [99]:
import sklearn
sklearn.metrics.confusion_matrix(Y_test, predicted)

array([[372,   9,   0,  10,   7,  33,   0,   0,   0,  29],
       [ 10, 432,   5,   0,  11,   3,  16,   0,  10,  15],
       [  0,  10, 424,   0,   0,   0,  40,   9,   4,   0],
       [ 27,   0,   0, 414,  17,  12,   0,   2,   2,   4],
       [  8,   1,   0,  58, 349,   3,   0,   7,   6,   2],
       [ 20,   0,   4,   3,   0, 407,   0,   1,   1,  10],
       [  1,  70, 177,   0,   2,   0, 168,   4,  17,   5],
       [  0,   2,   1,   2,   0,   2,   0, 408,   1,   2],
       [ 18,  17, 209,   3,  19,   1,  38,  28,  68,   2],
       [ 65,  21,   1,   0,   1,  21,   0,   1,   2, 295]], dtype=int64)